In [1]:
#!/usr/bin/env python
# coding: utf-8

### Importing required modules
import os
import sys

import time

import numpy as np
from scipy import signal
import pandas as pd

import matplotlib.pyplot as plt
from ipywidgets import interactive
import ipywidgets as widgets
import seaborn as sns
%matplotlib inline

# Change path
abspath = os.path.abspath("")
dname = os.path.dirname(abspath)
os.chdir(dname)
print("Directory changed to\n{}".format(dname))
sys.path.append('Brain-Signals/modules/')

import Functions as fnc

Directory changed to
C:\Users\HP\Desktop\Rescate\U\Decimo semestre\Practica profesional


In [2]:
### List of animals to process
IDs = ['ID1597', 'ID1659']
#, 'SERT1678', 'SERT1908', 'SERT1984', 'SERT1985', 'SERT2014', 'SERT1668', 'SERT1665', 'SERT2018', 'SERT2024', 'SERT2013']

### Filter parameters. You can add more frequency bands (lowcuts and highcuts in Hz)
filter_parameters = {'theta': {'N': 2, 'lowcut': 4,  'highcut': 20}} 

In [5]:
CohysID = dict()

for ID in IDs:
    print('\nLoading mouse {}...'.format(ID))
    Cohys = dict()
    for band in filter_parameters.keys():
        print(f"{band} Band ")
        epochs_dir = 'Brain-Signals/DATA/Cohy/'
        
        Cohys[band] = (np.load(f'{epochs_dir}{ID}_{band}_freq.npy'),\
        fnc.loadsplitdata(f'{epochs_dir}{ID}_{band}_Cohy.npy', 32),\
        np.load(f'Brain-Signals/DATA/Cohy/{ID}_{band}_Tfreq.npy'),\
        fnc.loadsplitdata(f'{epochs_dir}{ID}_{band}_TCohy.npy', 32))
         
        print(f"Stationary Coherency shape: {Cohys[band][1].shape}")
        print(f"Frequency shape: {Cohys[band][0].shape} Frequency max: {Cohys[band][0][-1]}")
        print(f"Time-dependent Coherency shape: {Cohys[band][3].shape}")
        print(f"Frequency domain shape {Cohys[band][2].shape} Frequency max: {Cohys[band][2][-1]}")
    CohysID[ID] = Cohys


Loading mouse ID1597...
theta Band 
Stationary Coherency shape: (4, 32, 32, 126)
Frequency shape: (126,) Frequency max: 20.0
Time-dependent Coherency shape: (4, 32, 32, 126, 123)
Frequency domain shape (126,) Frequency max: 20.0

Loading mouse ID1659...
theta Band 
Stationary Coherency shape: (9, 32, 32, 126)
Frequency shape: (126,) Frequency max: 20.0
Time-dependent Coherency shape: (9, 32, 32, 126, 123)
Frequency domain shape (126,) Frequency max: 20.0


# Gráficos para la Coherency

En esta sección se replicarán los gráficos utilizados en [Nolte2004](https://pubmed.ncbi.nlm.nih.gov/15351371/) y en [Adhikari2009](https://pubmed.ncbi.nlm.nih.gov/20152131/).

## Coh y iCoh estacionaria para una frecuencia fija

Para una determinada frecuencia graficaremos la coherencia y la parte imaginaría de la frecuencia entre los distintos canales como en las figuras 1,3 y 4 de Nolte2014

In [6]:
ID1 = IDs[1]
band = "theta"
Auxcohy1 = CohysID[ID1][band]

In [7]:
def Stat_cohy_f(nf, epoch):
    # Labels of matrix
    label_mconf = np.arange(CohysID[ID1][band][1].shape[1]) + 1 
    
    Coh = np.abs(CohysID[ID1][band][1][epoch,:,:,nf])
    iCoh = np.imag(CohysID[ID1][band][1][epoch,:,:,nf])
    f = CohysID[ID1][band][0]
    
    df_Coh = pd.DataFrame(Coh, label_mconf, label_mconf)
    df_iCoh = pd.DataFrame(iCoh, label_mconf, label_mconf)
    
    plt.figure(figsize=(25,10))
    plt.suptitle(f"Epoch {epoch+1} - Frequency = {f[nf]:.2f}\n\n", fontsize= "xx-large")
    plt.subplot(1,2,1)
    sns.set(font_scale=1) # for label size
    sns.heatmap(df_Coh, cmap="viridis", fmt=".2f") # Para anotar annot=True, annot_kws={"size": 10}
    
    plt.title(f"Coherence between channels\n", fontsize= "xx-large")
    plt.ylabel('Channels', fontsize = "x-large")
    plt.xlabel('Channels', fontsize = "x-large")
    
    plt.subplot(1,2,2)
    sns.set(font_scale=1) # for label size
    sns.heatmap(df_iCoh, cmap="viridis", fmt=".2f") # Para anotar annot=True, annot_kws={"size": 10}
    
    plt.title(f"Imaginary part of Coherency between channels\n", fontsize= "xx-large")
    plt.ylabel('Channels', fontsize = "x-large")
    plt.xlabel('Channels', fontsize = "x-large")
    
    plt.show()

In [8]:
iplt_stCohyf = interactive(Stat_cohy_f, nf = (0,Auxcohy1[0].shape[0]-1, 1), epoch =(0, Auxcohy1[1].shape[0]-1 , 1))

In [9]:
iplt_stCohyf

interactive(children=(IntSlider(value=62, description='nf', max=125), IntSlider(value=4, description='epoch', …

## Coh y iCoh estacionaria entre 2 canales: Dependencia de la frecuencia 

En esta sección se replicarán el gráfico C de la figura 1 de Adhikari2009

In [10]:
ID2 = IDs[0]
band = "theta"
Auxcohy2 = CohysID[ID2][band]

In [11]:
def Stat_cohy(ch1,ch2, epoch):
    # Labels of matrix
    
    Coh = np.abs(CohysID[ID2][band][1][epoch,ch1,ch2,:])
    iCoh = np.imag(CohysID[ID2][band][1][epoch,ch1,ch2,:])
    f = CohysID[ID2][band][0]
    
    plt.figure(figsize=(25,10))
    
    plt.suptitle(f"Epoch {epoch+1} - Channels {ch1+1} and {ch2+1}\n\n", fontsize= "xx-large")
    
    plt.subplot(1,2,1)
    plt.plot(f, Coh)
    
    plt.title(f"Coherence\n", fontsize= "xx-large")
    plt.ylabel('Coherence', fontsize = "x-large")
    plt.xlabel('Frequency', fontsize = "x-large")
    
    plt.subplot(1,2,2)
    plt.plot(f,iCoh)
    
    plt.title(f"Imaginary part of Coherency\n", fontsize= "xx-large")
    plt.ylabel('Im. of coherency', fontsize = "x-large")
    plt.xlabel('Frequency', fontsize = "x-large")
    
    plt.show()

In [12]:
iplt_stCohyf = interactive(Stat_cohy, ch1 = (0,Auxcohy2[1].shape[1]-1, 1), ch2 = (0,Auxcohy2[1].shape[1]-1, 1), epoch = (0,Auxcohy2[1].shape[0]-1, 1))
iplt_stCohyf 

interactive(children=(IntSlider(value=15, description='ch1', max=31), IntSlider(value=15, description='ch2', m…

## Evolución temporal de Coh y iCoh 

En este sección se replicarán el gráfico C de la figura 7 de Adhikari2009 (también figura 5 de Nolte2004)

In [13]:
ID = IDs[0]
band = "theta"
Auxcohy = CohysID[ID][band]

In [14]:
def Time_cohy(ch1,ch2, epoch):
    # Labels of matrix
    
    Coh = np.abs(CohysID[ID][band][3][epoch,ch1,ch2,:,:])
    iCoh = np.imag(CohysID[ID][band][3][epoch,ch1,ch2,:,:])
    
    df_Coh = pd.DataFrame(Coh)
    df_iCoh = pd.DataFrame(iCoh)
    
    plt.figure(figsize=(25,8))
    plt.suptitle(f"Epoch {epoch+1} - Channels {ch1+1} and {ch2+1}", fontsize= "xx-large", y=1.1)
    
    plt.subplot(1,2,1)
    #sns.set(font_scale=1) # for label size
    #sns.heatmap(df_Coh, fmt=".2f", xticklabels = False, yticklabels = False, vmin = Coh.min() , vmax= Coh.max()) # Para anotar annot=True, annot_kws={"size": 10}
    plt.imshow(Coh[::-1,:], cmap='turbo', interpolation = "mitchell")
    plt.colorbar()
    plt.title(f"Coherence between channels\n", fontsize= "xx-large")
    plt.ylabel('Frequency', fontsize = "x-large")
    plt.xlabel('Time', fontsize = "x-large")
    
    plt.subplot(1,2,2)
    #sns.set(font_scale=1) # for label size
    #sns.heatmap(df_iCoh, cmap="viridis", fmt=".2f", xticklabels = False, yticklabels = False) # Para anotar annot=True, annot_kws={"size": 10}
    
    #https://matplotlib.org/3.3.3/gallery/images_contours_and_fields/interpolation_methods.html#sphx-glr-gallery-images-contours-and-fields-interpolation-methods-py
    plt.imshow(iCoh[::-1,:], cmap='turbo', interpolation = "mitchell")
    plt.colorbar()

    plt.title(f"Imaginary part of Coherency between channels\n", fontsize= "xx-large")
    plt.ylabel('Frequency', fontsize = "x-large")
    plt.xlabel('Time', fontsize = "x-large")
    
    plt.show()

In [15]:
iplt_tCohyf = interactive(Time_cohy, ch1 = (0,Auxcohy[1].shape[1]-1, 1), ch2 = (0,Auxcohy[1].shape[1]-1, 1), epoch = (0,Auxcohy[1].shape[0]-1, 1))
iplt_tCohyf 

interactive(children=(IntSlider(value=15, description='ch1', max=31), IntSlider(value=15, description='ch2', m…